In [1766]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# plt.rcParams['figure.figsize'] = (10, 6)
import datetime as dt

import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
import numpy as np

from plotly.offline import iplot
import plotly
import plotly.graph_objs as go
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio
from plotly import tools

import os

# if not os.path.exists("images"):
#     os.mkdir("images")

In [1412]:
file = 'docs/metall_rates.csv'
df = pd.read_csv(file, 
                 decimal=',',
                sep=';')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 694 entries, 0 to 693
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    694 non-null    object 
 1   gold    694 non-null    object 
 2   silver  694 non-null    float64
dtypes: float64(1), object(2)
memory usage: 16.4+ KB


In [1413]:
df.head(5)

,date,gold,silver
0,31.10.2020,"4 784,4000",58.70
1,30.10.2020,"4 759,1700",60.67
2,29.10.2020,"4 729,5200",60.93
3,28.10.2020,"4 667,6900",59.68
4,27.10.2020,"4 673,6400",60.85


In [1414]:
df['gold'] = df['gold'].str.replace('\s','').str.replace(',','.')

In [1415]:
df['gold'] = pd.to_numeric(df['gold'])

In [1416]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 694 entries, 0 to 693
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   date    694 non-null    object 
 1   gold    694 non-null    float64
 2   silver  694 non-null    float64
dtypes: float64(2), object(1)
memory usage: 16.4+ KB


In [1417]:
df.head(10)

,date,gold,silver
0,31.10.2020,4784.40,58.70
1,30.10.2020,4759.17,60.67
2,29.10.2020,4729.52,60.93
3,28.10.2020,4667.69,59.68
4,27.10.2020,4673.64,60.85
5,24.10.2020,4697.13,60.87
6,23.10.2020,4750.35,61.74
7,22.10.2020,4752.55,61.97
8,21.10.2020,4767.05,61.59
9,20.10.2020,4785.15,62.23


In [1418]:
df['date'] = pd.to_datetime(df['date'],
                           format = '%d.%m.%Y')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 694 entries, 0 to 693
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    694 non-null    datetime64[ns]
 1   gold    694 non-null    float64       
 2   silver  694 non-null    float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 16.4 KB


In [1419]:
df.date = df.date.sort_values()
df.head()

,date,gold,silver
0,2020-10-31,4784.40,58.70
1,2020-10-30,4759.17,60.67
2,2020-10-29,4729.52,60.93
3,2020-10-28,4667.69,59.68
4,2020-10-27,4673.64,60.85


In [1420]:
# create date_range, end - 2020-10-23
yeardays = pd.date_range(start=pd.datetime(2018, 1, 1),
                           end=pd.datetime(2020, 11, 1))
yeardays

<ipython-input-1420-ef5feef068ad>:2: FutureWarning:

The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.

<ipython-input-1420-ef5feef068ad>:3: FutureWarning:

The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.



DatetimeIndex(['2018-01-01', '2018-01-02', '2018-01-03', '2018-01-04',
               '2018-01-05', '2018-01-06', '2018-01-07', '2018-01-08',
               '2018-01-09', '2018-01-10',
               ...
               '2020-10-23', '2020-10-24', '2020-10-25', '2020-10-26',
               '2020-10-27', '2020-10-28', '2020-10-29', '2020-10-30',
               '2020-10-31', '2020-11-01'],
              dtype='datetime64[ns]', length=1036, freq='D')

In [1421]:
# convert to DataFrame
yeardays = pd.DataFrame(yeardays)
yeardays = yeardays.rename(columns={0: "date"})
yeardays.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1036 entries, 0 to 1035
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    1036 non-null   datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 8.2 KB


In [1422]:
# merge two df's 
ndf = pd.merge(left=yeardays,
               right=df,
               on="date",
              how='outer')
ndf.head()

,date,gold,silver
0,2018-01-01,NaN,NaN
1,2018-01-02,NaN,NaN
2,2018-01-03,NaN,NaN
3,2018-01-04,NaN,NaN
4,2018-01-05,NaN,NaN


In [1423]:
ndf = ndf.ffill(axis = 0)
ndf.head(15)

,date,gold,silver
0,2018-01-01,NaN,NaN
1,2018-01-02,NaN,NaN
2,2018-01-03,NaN,NaN
3,2018-01-04,NaN,NaN
4,2018-01-05,NaN,NaN
5,2018-01-06,NaN,NaN
6,2018-01-07,NaN,NaN
7,2018-01-08,NaN,NaN
8,2018-01-09,NaN,NaN
9,2018-01-10,2411.72,31.49


In [1424]:
ndf = ndf.bfill(axis = 0)
ndf.head(15)

,date,gold,silver
0,2018-01-01,2411.72,31.49
1,2018-01-02,2411.72,31.49
2,2018-01-03,2411.72,31.49
3,2018-01-04,2411.72,31.49
4,2018-01-05,2411.72,31.49
5,2018-01-06,2411.72,31.49
6,2018-01-07,2411.72,31.49
7,2018-01-08,2411.72,31.49
8,2018-01-09,2411.72,31.49
9,2018-01-10,2411.72,31.49


In [1425]:
df = ndf
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1036 entries, 0 to 1035
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    1036 non-null   datetime64[ns]
 1   gold    1036 non-null   float64       
 2   silver  1036 non-null   float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 32.4 KB


In [1426]:
df['day'] = df['date'].dt.day
df['week'] = df['date'].dt.isocalendar().week
df['month'] = df['date'].dt.month

df.columns

Index(['date', 'gold', 'silver', 'day', 'week', 'month'], dtype='object')

In [1427]:
df.head()

,date,gold,silver,day,week,month
0,2018-01-01,2411.72,31.49,1,1,1
1,2018-01-02,2411.72,31.49,2,1,1
2,2018-01-03,2411.72,31.49,3,1,1
3,2018-01-04,2411.72,31.49,4,1,1
4,2018-01-05,2411.72,31.49,5,1,1


In [1428]:
# mean per week
df['mean_per_week_gold'] = df.groupby('week')['gold'].transform('mean')
df['mean_per_week_silver'] = df.groupby('week')['silver'].transform('mean')
df['mean_per_month_gold'] = df.groupby('month')['gold'].transform('mean')
df['mean_per_month_silver'] = df.groupby('month')['silver'].transform('mean')

In [1429]:
# rolling gold
df['rolling_7_gold'] = df['gold'].rolling(window=7).mean()
df['rolling_35_gold'] = df['gold'].rolling(window=35).mean()
df['rolling_50_gold'] = df['gold'].rolling(window=50).mean()

In [1430]:
# rolling silver
df['rolling_7_silver'] = df['silver'].rolling(window=7).mean()
df['rolling_35_silver'] = df['silver'].rolling(window=35).mean()
df['rolling_50_silver'] = df['silver'].rolling(window=50).mean()

In [1431]:
# exponential mean gold
df['ewm_7_gold'] = df['gold'].ewm(span=7, adjust=False).mean()
df['ewm_35_gold'] = df['gold'].ewm(span=35, adjust=False).mean()
df['ewm_50_gold'] = df['gold'].ewm(span=50, adjust=False).mean()

In [1432]:
# exponential mean silver
df['ewm_7_silver'] = df['silver'].ewm(span=7, adjust=False).mean()
df['ewm_35_silver'] = df['silver'].ewm(span=35, adjust=False).mean()
df['ewm_50_silver'] = df['silver'].ewm(span=50, adjust=False).mean()

In [1433]:
df.head(15)

,date,gold,silver,day,week,month,mean_per_week_gold,mean_per_week_silver,mean_per_month_gold,mean_per_month_silver,...,rolling_50_gold,rolling_7_silver,rolling_35_silver,rolling_50_silver,ewm_7_gold,ewm_35_gold,ewm_50_gold,ewm_7_silver,ewm_35_silver,ewm_50_silver
0,2018-01-01,2411.72,31.49,1,1,1,2764.575714,33.690476,2761.877527,33.423548,...,NaN,NaN,NaN,NaN,2411.720000,2411.720000,2411.720000,31.490000,31.490000,31.490000
1,2018-01-02,2411.72,31.49,2,1,1,2764.575714,33.690476,2761.877527,33.423548,...,NaN,NaN,NaN,NaN,2411.720000,2411.720000,2411.720000,31.490000,31.490000,31.490000
2,2018-01-03,2411.72,31.49,3,1,1,2764.575714,33.690476,2761.877527,33.423548,...,NaN,NaN,NaN,NaN,2411.720000,2411.720000,2411.720000,31.490000,31.490000,31.490000
3,2018-01-04,2411.72,31.49,4,1,1,2764.575714,33.690476,2761.877527,33.423548,...,NaN,NaN,NaN,NaN,2411.720000,2411.720000,2411.720000,31.490000,31.490000,31.490000
4,2018-01-05,2411.72,31.49,5,1,1,2764.575714,33.690476,2761.877527,33.423548,...,NaN,NaN,NaN,NaN,2411.720000,2411.720000,2411.720000,31.490000,31.490000,31.490000
5,2018-01-06,2411.72,31.49,6,1,1,2764.575714,33.690476,2761.877527,33.423548,...,NaN,NaN,NaN,NaN,2411.720000,2411.720000,2411.720000,31.490000,31.490000,31.490000
6,2018-01-07,2411.72,31.49,7,1,1,2764.575714,33.690476,2761.877527,33.423548,...,NaN,31.490000,NaN,NaN,2411.720000,2411.720000,2411.720000,31.490000,31.490000,31.490000
7,2018-01-08,2411.72,31.49,8,2,1,2755.680952,33.574762,2761.877527,33.423548,...,NaN,31.490000,NaN,NaN,2411.720000,2411.720000,2411.720000,31.490000,31.490000,31.490000
8,2018-01-09,2411.72,31.49,9,2,1,2755.680952,33.574762,2761.877527,33.423548,...,NaN,31.490000,NaN,NaN,2411.720000,2411.720000,2411.720000,31.490000,31.490000,31.490000
9,2018-01-10,2411.72,31.49,10,2,1,2755.680952,33.574762,2761.877527,33.423548,...,NaN,31.490000,NaN,NaN,2411.720000,2411.720000,2411.720000,31.490000,31.490000,31.490000


In [1434]:
df.index = df['date']
df.drop('date',axis = 1, inplace = True)
df.head()

,gold,silver,day,week,month,mean_per_week_gold,mean_per_week_silver,mean_per_month_gold,mean_per_month_silver,rolling_7_gold,...,rolling_50_gold,rolling_7_silver,rolling_35_silver,rolling_50_silver,ewm_7_gold,ewm_35_gold,ewm_50_gold,ewm_7_silver,ewm_35_silver,ewm_50_silver
date,,,,,,,,,,,,,,,,,,,,,
2018-01-01,2411.72,31.49,1,1,1,2764.575714,33.690476,2761.877527,33.423548,NaN,...,NaN,NaN,NaN,NaN,2411.72,2411.72,2411.72,31.49,31.49,31.49
2018-01-02,2411.72,31.49,2,1,1,2764.575714,33.690476,2761.877527,33.423548,NaN,...,NaN,NaN,NaN,NaN,2411.72,2411.72,2411.72,31.49,31.49,31.49
2018-01-03,2411.72,31.49,3,1,1,2764.575714,33.690476,2761.877527,33.423548,NaN,...,NaN,NaN,NaN,NaN,2411.72,2411.72,2411.72,31.49,31.49,31.49
2018-01-04,2411.72,31.49,4,1,1,2764.575714,33.690476,2761.877527,33.423548,NaN,...,NaN,NaN,NaN,NaN,2411.72,2411.72,2411.72,31.49,31.49,31.49
2018-01-05,2411.72,31.49,5,1,1,2764.575714,33.690476,2761.877527,33.423548,NaN,...,NaN,NaN,NaN,NaN,2411.72,2411.72,2411.72,31.49,31.49,31.49


In [1435]:
for i in df.columns:
    print(i)

gold
silver
day
week
month
mean_per_week_gold
mean_per_week_silver
mean_per_month_gold
mean_per_month_silver
rolling_7_gold
rolling_35_gold
rolling_50_gold
rolling_7_silver
rolling_35_silver
rolling_50_silver
ewm_7_gold
ewm_35_gold
ewm_50_gold
ewm_7_silver
ewm_35_silver
ewm_50_silver


In [1436]:
%%markdown

Установим цвета для последующего использования



Установим цвета для последующего использования


In [1437]:
# metall colors

# color_silver=['#1e3b6c', '#17386d', '#004c99', '#063c6e', '#0059b3', '#143474']
color_silver = '#0059b3'
# color_gold = ['#f9a612','#f3a211','#f3a21c','#f7a105']
color_gold = '#f3a211'

# produced / sell

color_produced = '#e25243'
# color_produced = '#481a52'
# color_produced = '#5d3465'

color_sell = '#087972'
# color_sell = '#54976c'
# color_sell = '#1c8c4b'

# trend, seasonal and etc.
color_trend = '#ec4b7c'
color_seasonal = '#2cbad3'
# color_ost = '#77539e'
color_ost = '#2d3852'


# bench
color_benchmark = '#99afbf'

In [1456]:
# Серебро и эмоциональная граница

fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, 
                         y=df.silver, 
                         name='Цена на серебро',
                         marker_color = color_silver))

fig.add_trace(go.Scatter(x=df.index, 
                         y=[35 for i in df.index], 
                         name='Эмоциональная граница',
                         marker_color = '#2abfbf',
                         line={'dash': 'dash'}))


fig.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
#     gridcolor='LightPink',
    autosize=False,
    width=1024,
    height=600,
    title="График цен на серебро c 01.01.2018 по 01.11.2020 год (цена дневная)",
    title_x = 0.5,
    legend=dict(x=.5, xanchor="center", orientation="h",
               font = dict(
#                    family = "Courier", 
                   size = 15, 
#                    color = "black"
               )),
    barmode='overlay',
#     margin=dict(l=50, r=50, t=30, b=30),
               )

fig.update_xaxes(showgrid=True, 
                 gridwidth=1, 
                 gridcolor='lightblue', 
                 title_text = 'месяц, год')
fig.update_yaxes(showgrid=True, 
                 gridwidth=1, 
                 gridcolor='lightblue',
                 title_text = 'рублей/грамм')


fig.show()

In [1464]:
# Золото и эмоциональная граница


fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, 
                         y=df.gold, 
                         name='Цена на золото',
                         marker_color = color_gold))

fig.add_trace(go.Scatter(x=df.index, 
                         y=[3000 for i in df.index], 
                         name='Эмоциональная граница',
                         marker_color = '#2abfbf',
                         line={'dash': 'dash'}))


fig.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    autosize=False,
    width=1024,
    height=600,
    title="График цен на золото c 01.01.2018 по 01.11.2020 год (цена дневная)",
    title_x = 0.5,
    legend=dict(x=.5, xanchor="center", orientation="h",
               font = dict(
#                    family = "Courier", 
                   size = 15, 
#                    color = "black"
               )),
    barmode='overlay',
#     margin=dict(l=50, r=50, t=30, b=30),
               )


fig.update_xaxes(showgrid=True, 
                 gridwidth=1, 
                 gridcolor='lightblue', 
                 title_text = 'месяц, год')
fig.update_yaxes(showgrid=True, 
                 gridwidth=1, 
                 gridcolor='lightblue',
                 title_text = 'рублей/грамм')


fig.show()

In [1468]:
# Серебро и средние

fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, 
                         y=df.silver, 
                         name='Цена на серебро',
                         marker_color = color_silver))
fig.add_trace(go.Scatter(x=df.index, 
                         y=df.rolling_50_silver, 
                         name='Скользящая средняя за 50 дней'))
fig.add_trace(go.Scatter(x=df.index, 
                         y=df.ewm_50_silver, 
                         name='Экспоненициальная скользящая средняя за 50 дней'))

fig.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
#     gridcolor='LightPink',
    autosize=False,
    width=1024,
    height=600,
    title="График цен на серебро c 01.01.2018 по 01.11.2020 год (цена дневная р/грамм и скользящие средние)",
    title_x = 0.5,
    legend=dict(x=.5, xanchor="center", orientation="h",
               font = dict(
#                    family = "Courier", 
                   size = 15, 
#                    color = "black"
               )),
    barmode='overlay',
#     margin=dict(l=50, r=50, t=30, b=30),
               )
                   
fig.update_xaxes(showgrid=True, 
                 gridwidth=1, 
                 gridcolor='lightblue', 
                 title_text = 'месяц, год')
fig.update_yaxes(showgrid=True, 
                 gridwidth=1, 
                 gridcolor='lightblue',
                 title_text = 'рублей/грамм')


fig.show()

In [1469]:
# Золото и средние

fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index, 
                         y=df.gold, 
                         name='Цена на золото',
                         marker_color = color_gold))
fig.add_trace(go.Scatter(
    x=df.index, 
    y=df.rolling_50_gold, 
    name='Скользящая средняя за 50 дней'))
fig.add_trace(go.Scatter(
    x=df.index, 
    y=df.ewm_50_gold, 
    name='Экспоненициальная скользящая средняя за 50 дней'))

fig.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    autosize=False,
    width=1024,
    height=600,
    title="График цен на золото c 01.01.2018 по 01.11.2020 год (цена дневная р/грамм и скользящие средние)",
    title_x = 0.5,
    legend=dict(x=.5, xanchor="center", orientation="h",
               font = dict(
#                    family = "Courier", 
                   size = 15, 
#                    color = "black"
               )),
    barmode='overlay',
#     margin=dict(l=50, r=50, t=30, b=30),
               )


fig.update_xaxes(showgrid=True, 
                 gridwidth=1, 
                 gridcolor='lightblue', 
                 title_text = 'месяц, год')
fig.update_yaxes(showgrid=True, 
                 gridwidth=1, 
                 gridcolor='lightblue',
                 title_text = 'рублей/грамм')


fig.show()

In [1472]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces

fig.add_trace(
    go.Scatter(x=df.index, 
               y=df.gold, 
               name="Золото",
               line=dict(
                   color=color_gold)
              ),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df.index, 
               y=df.silver, 
               name="Серебро",
               line=dict(
                   color=color_silver)
              ),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    title_text="Сравнение динамики цен на золото и серебро (2018 - 2020 год)",
    title_x = 0.47,
    width=1024,
    height=600,
    legend=dict(x=.5, xanchor="center", orientation="h",
               font = dict(
#                    family = "Courier", 
                   size = 15, 
#                    color = "black"
               )),
    barmode='overlay',
#     margin=dict(l=50, r=50, t=30, b=30),
               )

# Set x-axis title
fig.update_xaxes(title_text="<b>месяц, год</b>",
                 gridcolor='lightblue')

# Set y-axes titles
fig.update_yaxes(title_text="<b>Золото</b> (рублей/грамм)", 
                 secondary_y=False,
#                  gridcolor='#f5c541'
                )
fig.update_yaxes(title_text="<b>Серебро</b> (рублей/грамм)", 
                 secondary_y=True,
#                  gridcolor='#1167b1'
                )

fig.show()

In [1691]:
# Выпуск и реализация серебряных монет - данные

sell_silver = pd.DataFrame({'y': ['2018-07-01', '2019-07-01', '2020-07-01'], 
                          'Тираж': [150000, 100000, 300000],
                          'Реализовано': [47902, 64425, 174338]})
sell_silver['y'] = pd.to_datetime(sell_silver['y'],
                                  format = '%Y-%m-%d')
# sell_silver['y'] = sell_silver['y'].dt.year
sell_silver = sell_silver.set_index('y')
sell_silver['income']=0
sell_silver['income'][2]=135096
sell_silver['income'][1]=82725
sell_silver.head()

,Тираж,Реализовано,income
y,,,
2018-07-01,150000,47902,0
2019-07-01,100000,64425,82725
2020-07-01,300000,174338,135096


In [1692]:
# Выпуск и реализация серебряных монет - график


fig = go.Figure()

trace1 = go.Bar(x=sell_silver.index, # assign x as the dataframe column 'x'
                y=sell_silver['Реализовано'],
                name = 'Реализовано', 
                marker_color = color_sell,
#                 opacity=0.6,
               )


trace2 = go.Bar(x=sell_silver.index, # assign x as the dataframe column 'x'
                y=sell_silver['Тираж'],
                name = 'Тираж',
                marker_color = color_produced,
#                 opacity=0.6,
               )


fig.add_trace(trace1)
fig.add_trace(trace2)


fig.update_layout(
    barmode='group',
    plot_bgcolor='rgba(0,0,0,0)',
    title_text="Серебряные инвестиционные монеты (выпуск в обращение / реализация)",
    title_x = 0.5,
    width=1024,
    height=600,
    legend=dict(x=.5, xanchor="center", orientation="h",
               font = dict(
#                    family = "Courier", 
                   size = 15, 
#                    color = "black"
               )),
#     barmode='overlay',
#     margin=dict(l=50, r=50, t=30, b=30),
               )

# Set x-axis title
fig.update_xaxes(title_text="<b>Годы</b>",
                 gridcolor='lightblue')

# Set y-axes titles

fig.update_yaxes(title_text="<b>Цена на серебро</b> (р./грамм)", 
#                  secondary_y=True,
#                  gridcolor='#f5c541'
#                 side='left'
                )

fig.update_yaxes(title_text="<b>Шт.</b>", 
#                  secondary_y=False,
#                  gridcolor='#1167b1',
#                  side='right'
                )

fig.show()

In [1693]:
# Выпуск и реализация серебряных монет - график


fig = go.Figure()

trace0 = go.Bar(x=sell_silver.index, # assign x as the dataframe column 'x'
                y=sell_silver['income'],
                name = 'Входящий остаток',
#                 marker_color = color_produced,
#                 opacity=0.6,
               )


trace1 = go.Bar(x=sell_silver.index, # assign x as the dataframe column 'x'
                y=sell_silver['Реализовано'],
                name = 'Реализовано', 
                marker_color = color_sell,
#                 opacity=0.6,
               )


trace2 = go.Bar(x=sell_silver.index, # assign x as the dataframe column 'x'
                y=sell_silver['Тираж'],
                name = 'Тираж',
                marker_color = color_produced,
#                 opacity=0.6,
               )



fig.add_trace(trace0)
fig.add_trace(trace1)
fig.add_trace(trace2)


fig.update_layout(
    barmode='group',
    plot_bgcolor='rgba(0,0,0,0)',
    title_text="Серебряные инвестиционные монеты (выпуск в обращение / реализация)",
    title_x = 0.5,
    width=1024,
    height=600,
    legend=dict(x=.5, xanchor="center", orientation="h",
               font = dict(
#                    family = "Courier", 
                   size = 15, 
#                    color = "black"
               )),
#     barmode='overlay',
#     margin=dict(l=50, r=50, t=30, b=30),
               )

# Set x-axis title
fig.update_xaxes(title_text="<b>Годы</b>",
                 gridcolor='lightblue')

# Set y-axes titles

fig.update_yaxes(title_text="<b>Цена на серебро</b> (р./грамм)", 
#                  secondary_y=True,
#                  gridcolor='#f5c541'
#                 side='left'
                )

fig.update_yaxes(title_text="<b>Шт.</b>", 
#                  secondary_y=False,
#                  gridcolor='#1167b1',
#                  side='right'
                )

fig.show()

In [1694]:
# Выпуск и реализация золотыз монет - данные

sell_gold = pd.DataFrame({'y': ['2018-07-01', '2019-07-01', '2020-07-01'], 
                          'Тираж': [150000, 300000, 500000],
                          'Реализовано': [93367, 210172, 424876]})
sell_gold['y'] = pd.to_datetime(sell_gold['y'],
                                  format = '%Y-%m-%d')
# sell_gold['y'] = sell_gold['y'].dt.year
sell_gold = sell_gold.set_index('y')
# sell_gold.head()

sell_gold['income']=0
sell_gold['income'][2]=150834
sell_gold['income'][1]=54412
sell_gold.head()

,Тираж,Реализовано,income
y,,,
2018-07-01,150000,93367,0
2019-07-01,300000,210172,54412
2020-07-01,500000,424876,150834


In [1695]:
# Выпуск и реализация золотыз монет - график


fig = go.Figure()

trace1 = go.Bar(x=sell_gold.index, # assign x as the dataframe column 'x'
                y=sell_gold['Реализовано'],
                name = 'Реализовано', 
                marker_color = color_sell,
#                 opacity=0.6,
               )


trace2 = go.Bar(x=sell_gold.index, # assign x as the dataframe column 'x'
                y=sell_gold['Тираж'],
                name = 'Тираж',
                marker_color = color_produced,
#                 opacity=0.6,
               )


fig.add_trace(trace1)
fig.add_trace(trace2)


fig.update_layout(
    barmode='group',
    plot_bgcolor='rgba(0,0,0,0)',
    title_text="Золотые инвестиционные монеты (выпуск в обращение / реализация)",
    title_x = 0.5,
    width=1024,
    height=600,
    legend=dict(x=.5, xanchor="center", orientation="h",
               font = dict(
#                    family = "Courier", 
                   size = 15, 
#                    color = "black"
               )),
#     barmode='overlay',
#     margin=dict(l=50, r=50, t=30, b=30),
               )

# Set x-axis title
fig.update_xaxes(title_text="<b>Годы</b>",
                 gridcolor='lightblue')

# Set y-axes titles

fig.update_yaxes(title_text="<b>Цена на серебро</b> (р./грамм)", 
#                  secondary_y=True,
#                  gridcolor='#f5c541'
#                 side='left'
                )

fig.update_yaxes(title_text="<b>Шт.</b>", 
#                  secondary_y=False,
#                  gridcolor='#1167b1',
#                  side='right'
                )

fig.show()

In [1697]:
# Выпуск и реализация золотыз монет - график


fig = go.Figure()
trace0 = go.Bar(x=sell_gold.index, # assign x as the dataframe column 'x'
                y=sell_gold['income'],
                name = 'Входящий остаток', 
#                 marker_color = color_sell,
#                 opacity=0.6,
               )



trace1 = go.Bar(x=sell_gold.index, # assign x as the dataframe column 'x'
                y=sell_gold['Реализовано'],
                name = 'Реализовано', 
                marker_color = color_sell,
#                 opacity=0.6,
               )


trace2 = go.Bar(x=sell_gold.index, # assign x as the dataframe column 'x'
                y=sell_gold['Тираж'],
                name = 'Тираж',
                marker_color = color_produced,
#                 opacity=0.6,
               )

fig.add_trace(trace0)
fig.add_trace(trace1)
fig.add_trace(trace2)


fig.update_layout(
    barmode='group',
    plot_bgcolor='rgba(0,0,0,0)',
    title_text="Золотые инвестиционные монеты (выпуск в обращение / реализация)",
    title_x = 0.5,
    width=1024,
    height=600,
    legend=dict(x=.5, xanchor="center", orientation="h",
               font = dict(
#                    family = "Courier", 
                   size = 15, 
#                    color = "black"
               )),
#     barmode='overlay',
#     margin=dict(l=50, r=50, t=30, b=30),
               )

# Set x-axis title
fig.update_xaxes(title_text="<b>Годы</b>",
                 gridcolor='lightblue')

# Set y-axes titles

fig.update_yaxes(title_text="<b>Цена на серебро</b> (р./грамм)", 
#                  secondary_y=True,
#                  gridcolor='#f5c541'
#                 side='left'
                )

fig.update_yaxes(title_text="<b>Шт.</b>", 
#                  secondary_y=False,
#                  gridcolor='#1167b1',
#                  side='right'
                )

fig.show()

In [1485]:
invest_3_years = pd.DataFrame({'y': [2018, 2019, 2020], 
                               'Золото': [150000, 300000, 500000],
                              'Серебро': [150000, 100000, 300000]})
invest_3_years = invest_3_years.set_index('y')
# invest_3_years


# invest_3_years.index.unique()

# for i in invest_3_years.index.unique():
#     print(invest_3_years[invest_3_years.index == i])

# list(invest_3_years.columns)

# for i in invest_3_years.index.unique():
#     labels = list(invest_3_years.columns)
#     values = invest_3_years[invest_3_years.index == i].values[0]
#     print(labels, values)


metall_colors = [color_gold,color_silver]

for i in invest_3_years.index.unique():
    labels = list(invest_3_years.columns)
    
    values = invest_3_years[invest_3_years.index == i].values[0]
    
    fig = go.Figure([go.Pie(labels=labels, 
                            values=values, 
                            hole=.3,
                            marker_colors=metall_colors,
                            textinfo='label+percent')])
    
    fig.update_layout(title_text='Соотношение золотых / серебряных инвестиционных монет в шт., '+str(i)+' год',
                      title_x = 0.5,
                      showlegend=False)
    fig.show()

In [1728]:
invest_3_years = pd.DataFrame({'y': [2018, 2019, 2020], 
                               'Золото': [150000, 300000, 500000],
                              'Серебро': [150000, 100000, 300000]})
invest_3_years = invest_3_years.set_index('y')
invest_3_years['Золото, тонн'] = invest_3_years['Золото']*7.78/1000000
invest_3_years['Серебро, тонн'] = invest_3_years['Серебро']*31.1/1000000
invest_3_years.values[0][:2]

array([150000., 150000.])

In [1729]:
labels = list(invest_3_years[['Золото','Серебро']])
    
values = invest_3_years[invest_3_years.index == i].values[0]

metall_colors = [color_gold,color_silver]



fig = make_subplots(rows=1, 
                    cols=len(invest_3_years.index), 
                    specs=[[{'type':'pie'}, {'type':'pie'}, {'type':'pie'}]]
                   )

fig.add_trace(go.Pie(labels=labels, 
                     values=list(invest_3_years.values[0][:2]), 
                     name="2018 год",),
              row=1, col=1)
              

# fig.add_trace(go.Pie(values=[2, 3, 1]),
#               row=2, col=1)

fig.add_trace(go.Pie(labels=labels, 
                     values=list(invest_3_years.values[1][:2]), 
                     name="2019 год"),
              row=1, col=2)

fig.add_trace(go.Pie(labels=labels, 
                     values=list(invest_3_years.values[2][:2]), 
                     name="2020 год"),
              row=1, col=3)


# fig = go.Figure([go.Pie(labels=labels, 
#                         values=values, 
#                         hole=.3,
#                         marker_colors=metall_colors,
#                         textinfo='label+percent')])



# Use `hole` to create a donut-like pie chart
fig.update_traces(hole=.4, 
                  hoverinfo="label+percent+name",
                  marker_colors=metall_colors,
                  textinfo="label+percent+value",
                  insidetextorientation='radial')

fig.update_layout(
    title_text="Выпуск инвестиционных монет в 2018-2020 годах",
    width=950,
    height=450,
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='2018', 
                      x=0.11, 
#                       y=0.5, 
                      font_size=20, 
                      showarrow=False),
                 dict(text='2019', 
#                       x=0.82, 
#                       y=0.5, 
                      font_size=20, 
                      showarrow=False),
                 dict(text='2020', 
                      x=0.89, 
#                       y=0.5, 
                      font_size=20, 
                      showarrow=False)
                ]
)
fig.show()

In [1730]:
labels = list(invest_3_years[['Золото, тонн','Серебро, тонн']])
    
values = invest_3_years[invest_3_years.index == i].values[0]

metall_colors = [color_gold,color_silver]



fig = make_subplots(rows=1, 
                    cols=len(invest_3_years.index), 
                    specs=[[{'type':'pie'}, {'type':'pie'}, {'type':'pie'}]]
                   )

fig.add_trace(go.Pie(labels=labels, 
                     values=list(invest_3_years.values[0][2:]), 
                     name="2018 год",),
              row=1, col=1)
              

fig.add_trace(go.Pie(labels=labels, 
                     values=list(invest_3_years.values[1][2:]), 
                     name="2019 год"),
              row=1, col=2)

fig.add_trace(go.Pie(labels=labels, 
                     values=list(invest_3_years.values[2][2:]), 
                     name="2020 год"),
              row=1, col=3)


# fig = go.Figure([go.Pie(labels=labels, 
#                         values=values, 
#                         hole=.3,
#                         marker_colors=metall_colors,
#                         textinfo='label+percent')])



# Use `hole` to create a donut-like pie chart
fig.update_traces(hole=.4, 
                  hoverinfo="label+percent+name",
                  marker_colors=metall_colors,
                  textinfo="percent+value",
                  textposition='inside',
#                   insidetextorientation='radial'
                 )

fig.update_layout(
    title_text="Выпуск инвестиционных монет в 2018-2020 годах",
    width=950,
    height=450,
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='2018', 
                      x=0.11, 
#                       y=0.5, 
                      font_size=20, 
                      showarrow=False),
                 dict(text='2019', 
#                       x=0.82, 
#                       y=0.5, 
                      font_size=20, 
                      showarrow=False),
                 dict(text='2020', 
                      x=0.89, 
#                       y=0.5, 
                      font_size=20, 
                      showarrow=False)
                ]
)
fig.show()

In [ ]:
# Вторая часть аналитики

In [1553]:
def seasonal_decompose_plot(result, metall):
    if metall == 'золото':
        trace1 = go.Scatter(
            x=result.observed.index,
            y=result.observed,
            marker_color = color_gold
        )
    else:
        trace1 = go.Scatter(
            x=result.observed.index,
            y=result.observed,
            marker_color = color_silver
        )

    trace2 = go.Scatter(
        x=result.trend.index,
        y=result.trend,
        xaxis='x2',
        yaxis='y2',
        marker_color = color_trend
    )

    trace3 = go.Scatter(
        x=result.seasonal.index,
        y=result.seasonal,
        xaxis='x3',
        yaxis='y3',
        marker_color = color_seasonal
    )

    trace4 = go.Scatter(
        x=result.resid.index,
        y=result.resid,
        xaxis='x4',
        yaxis='y4',
        marker_color = color_ost
    )

    data = [trace1, trace2, trace3, trace4]
    layout = go.Layout(
        showlegend=False,
    )

    fig = make_subplots(
        rows=4,
        cols=1,
        print_grid=False
    )

    fig.append_trace(trace1, 1, 1)
    fig.append_trace(trace2, 2, 1)
    fig.append_trace(trace3, 3, 1)
    fig.append_trace(trace4, 4, 1)

    fig['layout'].update(
        plot_bgcolor='rgba(0,0,0,0)',
        title=(f"Декомпозиция временного ряда 2020 ({metall})"),
        title_x = 0.5,
        showlegend=False,
        height=1024,
        yaxis=dict(title="График",
                   gridcolor='lightblue'),
        xaxis = dict(gridcolor='lightblue'),

        yaxis2=dict(title="Тренд",
                   gridcolor='lightblue'),
        xaxis2 = dict(gridcolor='lightblue'),

        yaxis3=dict(title="Сезонная компонента",
                   gridcolor='lightblue'),
        xaxis3 = dict(gridcolor='lightblue'),

        yaxis4=dict(title="Остаточная компонента",
                   gridcolor='lightblue'),
        xaxis4=dict(title="Временной ряд",
                    gridcolor='lightblue'))


    plotly.offline.iplot(fig)

In [1554]:
start_01_01_2020 = pd.to_datetime("2020-01-01").date()
df2020 = df[start_01_01_2020:]
# df_last_month = df[last_month:]
df2020.tail()

,gold,silver,day,week,month,mean_per_week_gold,mean_per_week_silver,mean_per_month_gold,mean_per_month_silver,rolling_7_gold,...,rolling_50_gold,rolling_7_silver,rolling_35_silver,rolling_50_silver,ewm_7_gold,ewm_35_gold,ewm_50_gold,ewm_7_silver,ewm_35_silver,ewm_50_silver
date,,,,,,,,,,,,,,,,,,,,,
2020-10-28,4667.69,59.68,28,44,10,3469.700952,42.575238,3472.463978,42.657204,4705.088571,...,4721.2768,60.978571,59.923143,61.3794,4700.706835,4723.267334,4706.138732,60.724601,60.969275,60.858091
2020-10-29,4729.52,60.93,29,44,10,3469.700952,42.575238,3472.463978,42.657204,4701.798571,...,4722.0530,60.830000,60.009143,61.2950,4707.910126,4723.614704,4707.055645,60.775951,60.967093,60.860911
2020-10-30,4759.17,60.67,30,44,10,3469.700952,42.575238,3472.463978,42.657204,4703.058571,...,4722.9088,60.677143,60.167143,61.2104,4720.725095,4725.589998,4709.099345,60.749463,60.950588,60.853424
2020-10-31,4784.40,58.70,31,44,10,3469.700952,42.575238,3472.463978,42.657204,4715.525714,...,4724.1472,60.367143,60.247429,61.0680,4736.643821,4728.857220,4712.052312,60.237097,60.825556,60.768976
2020-11-01,4784.40,58.70,1,44,11,3469.700952,42.575238,2847.372623,33.489508,4727.992857,...,4726.1976,60.057143,60.327714,60.9478,4748.582866,4731.942930,4714.889476,59.852823,60.707469,60.687840


In [1555]:
df2020.head()

,gold,silver,day,week,month,mean_per_week_gold,mean_per_week_silver,mean_per_month_gold,mean_per_month_silver,rolling_7_gold,...,rolling_50_gold,rolling_7_silver,rolling_35_silver,rolling_50_silver,ewm_7_gold,ewm_35_gold,ewm_50_gold,ewm_7_silver,ewm_35_silver,ewm_50_silver
date,,,,,,,,,,,,,,,,,,,,,
2020-01-01,3031.25,35.48,1,1,1,2764.575714,33.690476,2761.877527,33.423548,2999.495714,...,2998.6758,35.140000,34.589429,34.6830,3005.450497,2996.990214,3005.292707,35.121478,34.769384,34.904633
2020-01-02,3031.25,35.48,2,1,1,2764.575714,33.690476,2761.877527,33.423548,3009.934286,...,2999.5608,35.224286,34.608000,34.6996,3011.900373,2998.893536,3006.310640,35.211109,34.808863,34.927197
2020-01-03,3031.25,35.48,3,1,1,2764.575714,33.690476,2761.877527,33.423548,3020.021429,...,2999.7716,35.304286,34.617429,34.7184,3016.737780,3000.691117,3007.288654,35.278331,34.846148,34.948875
2020-01-04,3031.25,35.48,4,1,1,2764.575714,33.690476,2761.877527,33.423548,3022.674286,...,2999.8002,35.362857,34.633714,34.7282,3020.365835,3002.388833,3008.228314,35.328749,34.881362,34.969704
2020-01-05,3031.25,35.48,5,1,1,2764.575714,33.690476,2761.877527,33.423548,3025.327143,...,3000.2170,35.421429,34.650000,34.7380,3023.086876,3003.992231,3009.131126,35.366561,34.914620,34.989715


In [1556]:
df2020.sort_index(inplace=True)
df2020.head()

,gold,silver,day,week,month,mean_per_week_gold,mean_per_week_silver,mean_per_month_gold,mean_per_month_silver,rolling_7_gold,...,rolling_50_gold,rolling_7_silver,rolling_35_silver,rolling_50_silver,ewm_7_gold,ewm_35_gold,ewm_50_gold,ewm_7_silver,ewm_35_silver,ewm_50_silver
date,,,,,,,,,,,,,,,,,,,,,
2020-01-01,3031.25,35.48,1,1,1,2764.575714,33.690476,2761.877527,33.423548,2999.495714,...,2998.6758,35.140000,34.589429,34.6830,3005.450497,2996.990214,3005.292707,35.121478,34.769384,34.904633
2020-01-02,3031.25,35.48,2,1,1,2764.575714,33.690476,2761.877527,33.423548,3009.934286,...,2999.5608,35.224286,34.608000,34.6996,3011.900373,2998.893536,3006.310640,35.211109,34.808863,34.927197
2020-01-03,3031.25,35.48,3,1,1,2764.575714,33.690476,2761.877527,33.423548,3020.021429,...,2999.7716,35.304286,34.617429,34.7184,3016.737780,3000.691117,3007.288654,35.278331,34.846148,34.948875
2020-01-04,3031.25,35.48,4,1,1,2764.575714,33.690476,2761.877527,33.423548,3022.674286,...,2999.8002,35.362857,34.633714,34.7282,3020.365835,3002.388833,3008.228314,35.328749,34.881362,34.969704
2020-01-05,3031.25,35.48,5,1,1,2764.575714,33.690476,2761.877527,33.423548,3025.327143,...,3000.2170,35.421429,34.650000,34.7380,3023.086876,3003.992231,3009.131126,35.366561,34.914620,34.989715


In [1557]:
# result = seasonal_decompose(df2020['gold'], model='additive')
# seasonal_decompose_plot(result, 'золото')

In [1558]:
result = seasonal_decompose(df2020['gold'], model='multiplicative')
seasonal_decompose_plot(result, 'золото')

In [1560]:
gold_trend = pd.DataFrame(result.trend)
gold_trend

,trend
date,
2020-01-01,NaN
2020-01-02,NaN
2020-01-03,NaN
2020-01-04,3031.250000
2020-01-05,3031.250000
...,...
2020-10-28,4715.525714
2020-10-29,4727.992857
2020-10-30,NaN


In [1561]:
result = seasonal_decompose(df2020['silver'], model='multiplicative')
seasonal_decompose_plot(result, 'серебро')

In [1562]:
silver_trend = pd.DataFrame(result.trend)
silver_trend

,trend
date,
2020-01-01,NaN
2020-01-02,NaN
2020-01-03,NaN
2020-01-04,35.480000
2020-01-05,35.480000
...,...
2020-10-28,60.367143
2020-10-29,60.057143
2020-10-30,NaN


In [1542]:
# тут построить разложения по месяцам

In [1543]:
last_month = pd.to_datetime("2020-10-01").date()

In [1544]:
df.tail()

,gold,silver,day,week,month,mean_per_week_gold,mean_per_week_silver,mean_per_month_gold,mean_per_month_silver,rolling_7_gold,...,rolling_50_gold,rolling_7_silver,rolling_35_silver,rolling_50_silver,ewm_7_gold,ewm_35_gold,ewm_50_gold,ewm_7_silver,ewm_35_silver,ewm_50_silver
date,,,,,,,,,,,,,,,,,,,,,
2020-10-28,4667.69,59.68,28,44,10,3469.700952,42.575238,3472.463978,42.657204,4705.088571,...,4721.2768,60.978571,59.923143,61.3794,4700.706835,4723.267334,4706.138732,60.724601,60.969275,60.858091
2020-10-29,4729.52,60.93,29,44,10,3469.700952,42.575238,3472.463978,42.657204,4701.798571,...,4722.0530,60.830000,60.009143,61.2950,4707.910126,4723.614704,4707.055645,60.775951,60.967093,60.860911
2020-10-30,4759.17,60.67,30,44,10,3469.700952,42.575238,3472.463978,42.657204,4703.058571,...,4722.9088,60.677143,60.167143,61.2104,4720.725095,4725.589998,4709.099345,60.749463,60.950588,60.853424
2020-10-31,4784.40,58.70,31,44,10,3469.700952,42.575238,3472.463978,42.657204,4715.525714,...,4724.1472,60.367143,60.247429,61.0680,4736.643821,4728.857220,4712.052312,60.237097,60.825556,60.768976
2020-11-01,4784.40,58.70,1,44,11,3469.700952,42.575238,2847.372623,33.489508,4727.992857,...,4726.1976,60.057143,60.327714,60.9478,4748.582866,4731.942930,4714.889476,59.852823,60.707469,60.687840


In [1545]:
df_last_month = df[last_month:]
df_last_month.tail(35)

,gold,silver,day,week,month,mean_per_week_gold,mean_per_week_silver,mean_per_month_gold,mean_per_month_silver,rolling_7_gold,...,rolling_50_gold,rolling_7_silver,rolling_35_silver,rolling_50_silver,ewm_7_gold,ewm_35_gold,ewm_50_gold,ewm_7_silver,ewm_35_silver,ewm_50_silver
date,,,,,,,,,,,,,,,,,,,,,
2020-10-01,4770.62,60.10,1,40,10,3463.947143,42.108571,3472.463978,42.657204,4674.731429,...,4666.2600,57.410000,63.377429,63.4390,4711.335791,4672.251457,4639.716604,59.010317,61.555421,61.026525
2020-10-02,4709.54,58.68,2,40,10,3463.947143,42.108571,3472.463978,42.657204,4691.477143,...,4669.4846,57.915714,63.171143,63.3972,4710.886843,4674.323043,4642.454776,58.927738,61.395675,60.934504
2020-10-03,4786.39,59.92,3,40,10,3463.947143,42.108571,3472.463978,42.657204,4715.440000,...,4673.8178,58.491429,63.008000,63.3532,4729.762632,4680.548985,4648.099295,59.175803,61.313693,60.894719
2020-10-04,4786.39,59.92,4,40,10,3463.947143,42.108571,3472.463978,42.657204,4739.402857,...,4677.8220,59.067143,62.844857,63.2944,4743.919474,4686.429041,4653.522460,59.361853,61.236266,60.856495
2020-10-05,4786.39,59.92,5,41,10,3484.676667,42.569524,3472.463978,42.657204,4763.365714,...,4681.8262,59.642857,62.681714,63.2356,4754.537106,4691.982428,4658.732951,59.501389,61.163140,60.819770
2020-10-06,4771.68,60.07,6,41,10,3484.676667,42.569524,3472.463978,42.657204,4776.222857,...,4685.5362,59.952857,62.543714,63.1798,4758.822829,4696.410071,4663.162247,59.643542,61.102410,60.790367
2020-10-07,4827.56,61.34,7,41,10,3484.676667,42.569524,3472.463978,42.657204,4776.938571,...,4690.6020,59.992857,62.343714,63.1438,4776.007122,4703.696178,4669.609218,60.067657,61.115610,60.811921
2020-10-08,4740.23,59.08,8,41,10,3484.676667,42.569524,3472.463978,42.657204,4772.597143,...,4690.7276,59.847143,62.152000,62.9880,4767.062841,4705.725835,4672.378661,59.820742,61.002520,60.744003
2020-10-09,4737.92,60.02,9,41,10,3484.676667,42.569524,3472.463978,42.657204,4776.651429,...,4691.6208,60.038571,62.002000,62.8962,4759.777131,4707.514399,4674.948909,59.870557,60.947936,60.715611


In [1546]:
result = seasonal_decompose(df_last_month['silver'], model='multiplicative')
seasonal_decompose_plot(result, 'серебро')

In [1547]:
result = seasonal_decompose(df_last_month['gold'], model='multiplicative')
seasonal_decompose_plot(result, 'золото')

In [1550]:
last_two_months = pd.to_datetime("2020-09-01").date()
df_last_two_months = df[last_two_months:]

In [1551]:
result = seasonal_decompose(df_last_two_months['silver'], model='multiplicative')
seasonal_decompose_plot(result, 'серебро')

In [1552]:
result = seasonal_decompose(df_last_two_months['gold'], model='multiplicative')
seasonal_decompose_plot(result, 'золото')

In [1616]:
deposit_rates = pd.read_csv('docs/depos.csv', sep=';', decimal=',')
deposit_rates = deposit_rates[['декада', 'Ставка']]
deposit_rates = deposit_rates.rename(columns={'Ставка':'rate', 'декада':'date'})

deposit_rates['date'][23]='29.02.2020'
deposit_rates['date'] = deposit_rates['date'].str.replace('III', '30').str.replace('II', '20').str.replace('I', '10')

deposit_rates['date'] = pd.to_datetime(deposit_rates['date'],format = '%d.%m.%Y')
deposit_rates = deposit_rates.set_index('date')

deposit_rates

<ipython-input-1616-6cfcae3aacaf>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,rate
date,
2020-10-20,4.369
2020-10-10,4.326
2020-09-30,4.396
2020-09-20,4.396
2020-09-10,4.406
2020-08-30,4.425
2020-08-20,4.450
2020-08-10,4.515
2020-07-30,4.584


In [1652]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Create traces
trace0 = go.Scatter(
    x=gold_trend.index,
    y=gold_trend.trend,
    xaxis='x1',
    yaxis='y1',
    marker_color = color_trend,
    name = 'Тренд золота'
)

trace1 = go.Scatter(
    x=deposit_rates.index,
    y=deposit_rates.rate,
    xaxis='x1',
    yaxis='y2',
    marker_color = color_benchmark,
    name = 'Ставки по депозитам'
)


# Add traces
fig.add_trace(trace0, secondary_y=False)
fig.add_trace(trace1, secondary_y=True)


# Add figure title
fig.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    title_text="Сравнение тренда учетных цен на золото и депозитных ставок",
    title_x = 0.47,
    width=1024,
    height=600,
    legend=dict(x=.5, xanchor="center", orientation="h",
               font = dict(
#                    family = "Courier", 
                   size = 15, 
#                    color = "black"
               )),
    barmode='overlay',
#     margin=dict(l=50, r=50, t=30, b=30),
               )



# Set x-axis title
fig.update_xaxes(title_text="<b>месяц, 2020 год</b>",
                 gridcolor='lightblue')

# Set y-axes titles
fig.update_yaxes(title_text="<b>Золото</b> (рублей/грамм)", 
                 secondary_y=False,
#                  gridcolor='#f5c541'
                )
fig.update_yaxes(title_text="<b>Депозит</b> (процентная ставка)", 
                 secondary_y=True,
#                  gridcolor='#1167b1'
                )

fig.show()

In [1756]:
gold_deposit_df = gold_trend.merge(deposit_rates, left_index=True, right_index=True, how='left')
gold_deposit_df.head(15)

,trend,rate
date,,
2020-01-01,NaN,NaN
2020-01-02,NaN,NaN
2020-01-03,NaN,NaN
2020-01-04,3031.250000,NaN
2020-01-05,3031.250000,NaN
2020-01-06,3031.250000,NaN
2020-01-07,3033.540000,NaN
2020-01-08,3036.304286,NaN
2020-01-09,3039.068571,NaN


In [1757]:
gold_deposit_df_pct = gold_deposit_df.pct_change()

In [1760]:
gold_deposit_corr = gold_deposit_df.dropna().trend.corr(gold_deposit_df.dropna().rate
                                                           )
"Correlation beetween gold/deposites is {:.3}".format(gold_deposit_corr)

'Correlation beetween gold/deposites is -0.877'

In [1761]:
gold_deposit_pct_corr = gold_deposit_df.dropna().trend.pct_change().corr(
    gold_deposit_df.dropna().rate.pct_change()
)

"Percent change correlation is {:.3}".format(gold_deposit_pct_corr)

'Percent change correlation is 0.204'

In [1653]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Create traces
trace0 = go.Scatter(
    x=silver_trend.index,
    y=silver_trend.trend,
    xaxis='x1',
    yaxis='y1',
    marker_color = color_trend,
    name = 'Тренд серебра'
)

trace1 = go.Scatter(
    x=deposit_rates.index,
    y=deposit_rates.rate,
    xaxis='x1',
    yaxis='y2',
    marker_color = color_benchmark,
    name = 'Ставки по депозитам'
)


# Add traces
fig.add_trace(trace0, secondary_y=False)
fig.add_trace(trace1, secondary_y=True)


# Add figure title
fig.update_layout(
    plot_bgcolor='rgba(0,0,0,0)',
    title_text="Сравнение тренда учетных цен на серебро и депозитных ставок",
    title_x = 0.47,
    width=1024,
    height=600,
    legend=dict(x=.5, xanchor="center", orientation="h",
               font = dict(
#                    family = "Courier", 
                   size = 15, 
#                    color = "black"
               )),
    barmode='overlay',
#     margin=dict(l=50, r=50, t=30, b=30),
               )



# Set x-axis title
fig.update_xaxes(title_text="<b>месяц, 2020 год</b>",
                 gridcolor='lightblue')

# Set y-axes titles
fig.update_yaxes(title_text="<b>Серебро</b> (рублей/грамм)", 
                 secondary_y=False,
#                  gridcolor='#f5c541'
                )
fig.update_yaxes(title_text="<b>Депозит</b> (процентная ставка)", 
                 secondary_y=True,
#                  gridcolor='#1167b1'
                )

fig.show()

In [1769]:
# Add a constant to the DataFrame x
reg_def = sm.add_constant(gold_deposit_df.dropna())

# Fit the regression of y on x
result = sm.OLS(reg_def.trend, reg_def[['const', 'rate']]).fit()

# Print out the results
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  trend   R-squared:                       0.770
Model:                            OLS   Adj. R-squared:                  0.761
Method:                 Least Squares   F-statistic:                     90.34
Date:                Tue, 03 Nov 2020   Prob (F-statistic):           4.17e-10
Time:                        12:57:18   Log-Likelihood:                -202.39
No. Observations:                  29   AIC:                             408.8
Df Residuals:                      27   BIC:                             411.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       8833.8447    506.957     17.425      0.000    7793.654    9874.035
rate        -956.1963    100.601     -9.505      0.000   -1162.612    -749.781
==============================================================================
Omnibus:                        4.228   Durbin-Watson:                   0.500
Prob(Omnibus):                  0.121   Jarque-Bera (JB):                1.906
Skew:                           0.290   Prob(JB):                        0.386
Kurtosis:                       1.886   Cond. No.                         53.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

В качестве зависимой переменной выступает trend - цена на металл.  
Регрессионная модель показывает, что два временных ряда значимо связаны (rate P>|t| < 0.05), причём ставка по депозиту объясняет 70% дисперсии цены на металл (R-squared: 0.770).
Исходя из этого мы можем сказать, что связь между этими показателями определённо есть.

In [1774]:
# автокорреляции металлов и трендов

"Price metall autocorrelation {:.3}, Percent Change autocorrelation {:.3}".format(
    gold_deposit_df.trend.autocorr(),
    gold_deposit_df.trend.pct_change().autocorr()
)

'Price metall autocorrelation 1.0, Percent Change autocorrelation 0.833'

In [1775]:
"Deposit rates autocorrelation {:.3}, Percent Change autocorrelation {:.3}".format(
    gold_deposit_df.dropna().rate.autocorr(),
    gold_deposit_df.rate.dropna().pct_change().autocorr()
)

'Deposit rates autocorrelation 0.972, Percent Change autocorrelation 0.181'

In [1648]:
silver_deposit_df = silver_trend.merge(deposit_rates, left_index=True, right_index=True, how='left')
silver_deposit_df.head(15)

,trend,rate
date,,
2020-01-01,NaN,NaN
2020-01-02,NaN,NaN
2020-01-03,NaN,NaN
2020-01-04,35.480000,NaN
2020-01-05,35.480000,NaN
2020-01-06,35.480000,NaN
2020-01-07,35.591429,NaN
2020-01-08,35.562857,NaN
2020-01-09,35.534286,NaN


In [1650]:
silver_deposit_corr = silver_deposit_df.dropna().trend.corr(silver_deposit_df.dropna().rate)
"Correlation beetween silver/deposites is {:.3}".format(silver_deposit_corr)

'Correlation beetween silver/deposites is -0.858'

In [1651]:
fig = go.Figure(data=go.Splom(
                dimensions=[dict(label='silver trend',
                                 values=silver_deposit_df.dropna().trend),
                            dict(label='deposites',
                                 values=silver_deposit_df.dropna().rate)],
#                 text=df['class'],
                marker=dict(
#                     color=index_vals,
                            showscale=False, # colors encode categorical variables
                            line_color='white', line_width=0.5)
                )
               )


fig.update_layout(
    title='Gold/Deposit',
    width=600,
    height=600,
)

fig.show()

In [1384]:
# разбить внизу график на 3 года:
# https://community.plotly.com/t/multiple-x-axes-with-different-ticks-depending-on-zoom-level/38634

In [1386]:
# import datetime as dt, numpy as np, plotly.graph_objs as go
# from scipy.stats import norm
# from calendar import monthrange


# nCategory = 5
# nDays = 100

# # generate bar data
# base = dt.datetime.combine(dt.date.today(),dt.time(hour=12,minute=0))
# date_list = [base + dt.timedelta(days=x) for x in range(nDays)]

# barData = []
# categoryLabels = []
# barChart = []
# for i in range(nCategory):
#     barData = norm.rvs(4,1,nDays).tolist()
#     barChart.append(go.Bar(x=date_list,y=barData, xaxis = 'x1',name = str(i), hovertemplate = '<b>%{x|%x}<br>%{y}<br></b>'))

# xaxis1_vals = date_list
# xaxis1_text = ['' for d in date_list]
# xaxis1 = go.layout.XAxis(tickvals = xaxis1_vals,ticktext = xaxis1_text, showgrid = False)

# # x-axis for the week numbers
# dummyScatter1 = go.Scatter(x=date_list,y=[-1]*nDays, xaxis = 'x2',showlegend = False,hoverinfo='skip')
# xaxis2_vals = [d for d in date_list if d.isoweekday()==3]
# xaxis2 = go.layout.XAxis(matches = "x1",anchor="x1",overlaying="x1", 
#                   tickcolor = 'rgba(0,0,0,0)', tickformat = '%U',
#                   tickvals = xaxis2_vals, showgrid = False)

# # x-axis for the dividers between weeks
# dummyScatter2 = go.Scatter(x=date_list,y=[-1]*nDays, xaxis = 'x3',showlegend = False,hoverinfo='skip')
# xaxis3_vals = [d.date() for d in date_list if d.isoweekday()==7]
# xaxis3_text = ['']*len(xaxis3_vals)
# xaxis3 = go.layout.XAxis(matches = "x1",anchor="x1",overlaying="x1",ticklen = 10, 
#                   ticks= 'outside',tickvals = xaxis3_vals, ticktext = xaxis3_text, showgrid = False, gridwidth=5)

# # x-axis for the month names
# dummyScatter3 = go.Scatter(x=date_list,y=[-1]*nDays, xaxis = 'x4',showlegend = False,hoverinfo='skip')
# xaxis4_vals = [] 
# for d in date_list:
#     if d.day == np.floor(monthrange(d.year,d.month)[1]/2):
#         xaxis4_vals.append(d + dt.timedelta(days=np.mod(monthrange(d.year,d.month)[1],2)/2))
# xaxis4 = go.layout.XAxis(matches = "x1",anchor="x1",overlaying="x1", 
#                   tickcolor = 'rgba(0,0,0,0)', tickformat = '%B',
#                   tickvals = xaxis4_vals, ticks= 'outside', ticklen = 30, showgrid = False)

# # x-axis for the dividers between months
# dummyScatter4 = go.Scatter(x=date_list,y=[-1]*nDays, xaxis = 'x5',showlegend = False,hoverinfo='skip')
# xaxis5_vals = [d.date() for d in date_list if d.day==monthrange(d.year,d.month)[1]]
# xaxis5_text = ['']*len(xaxis5_vals)
# xaxis5 = go.layout.XAxis(matches = "x1",anchor="x1",overlaying="x1",ticklen = 50, 
#                   ticks= 'outside', tickvals = xaxis5_vals, ticktext = xaxis5_text, showgrid = True)

# yaxis = go.layout.YAxis(rangemode = 'nonnegative')
# layout = go.Layout(xaxis=dict(), barmode = 'stack', xaxis1 = xaxis1, xaxis2 = xaxis2, 
#                   xaxis3 = xaxis3, xaxis4 = xaxis4, xaxis5 = xaxis5, yaxis = yaxis, hovermode = 'x')
# fig = go.Figure(data=barChart + [dummyScatter1,dummyScatter2,dummyScatter3,dummyScatter4],layout=layout)

# fig.show()

In [1387]:
# test df

# testdate = pd.date_range(start=pd.datetime(2018, 1, 1),
#                            end=pd.datetime(2020, 11, 1))
# testdate = pd.DataFrame(testdate)
# testdate['price'] = np.random.randint(30, 50, testdate.shape[0])
# testdate = testdate.rename(columns={0: "date"})
# testdate = testdate.set_index('date')
# testdate

In [1388]:
df2020

,gold,silver,day,week,month,mean_per_week_gold,mean_per_week_silver,mean_per_month_gold,mean_per_month_silver,rolling_7_gold,...,rolling_50_gold,rolling_7_silver,rolling_35_silver,rolling_50_silver,ewm_7_gold,ewm_35_gold,ewm_50_gold,ewm_7_silver,ewm_35_silver,ewm_50_silver
date,,,,,,,,,,,,,,,,,,,,,
2020-01-01,3031.25,35.48,1,1,1,2764.575714,33.690476,2761.877527,33.423548,2999.495714,...,2998.6758,35.140000,34.589429,34.6830,3005.450497,2996.990214,3005.292707,35.121478,34.769384,34.904633
2020-01-02,3031.25,35.48,2,1,1,2764.575714,33.690476,2761.877527,33.423548,3009.934286,...,2999.5608,35.224286,34.608000,34.6996,3011.900373,2998.893536,3006.310640,35.211109,34.808863,34.927197
2020-01-03,3031.25,35.48,3,1,1,2764.575714,33.690476,2761.877527,33.423548,3020.021429,...,2999.7716,35.304286,34.617429,34.7184,3016.737780,3000.691117,3007.288654,35.278331,34.846148,34.948875
2020-01-04,3031.25,35.48,4,1,1,2764.575714,33.690476,2761.877527,33.423548,3022.674286,...,2999.8002,35.362857,34.633714,34.7282,3020.365835,3002.388833,3008.228314,35.328749,34.881362,34.969704
2020-01-05,3031.25,35.48,5,1,1,2764.575714,33.690476,2761.877527,33.423548,3025.327143,...,3000.2170,35.421429,34.650000,34.7380,3023.086876,3003.992231,3009.131126,35.366561,34.914620,34.989715
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-10-28,4667.69,59.68,28,44,10,3469.700952,42.575238,3472.463978,42.657204,4705.088571,...,4721.2768,60.978571,59.923143,61.3794,4700.706835,4723.267334,4706.138732,60.724601,60.969275,60.858091
2020-10-29,4729.52,60.93,29,44,10,3469.700952,42.575238,3472.463978,42.657204,4701.798571,...,4722.0530,60.830000,60.009143,61.2950,4707.910126,4723.614704,4707.055645,60.775951,60.967093,60.860911
2020-10-30,4759.17,60.67,30,44,10,3469.700952,42.575238,3472.463978,42.657204,4703.058571,...,4722.9088,60.677143,60.167143,61.2104,4720.725095,4725.589998,4709.099345,60.749463,60.950588,60.853424


In [1389]:
# Расчет ежемесячных процентов

from dateutil import relativedelta as rdelta
from datetime import date

d1 = date(2020,1,1)
d2 = date(2020,11,1)
m = rdelta.relativedelta(d2,d1).months
x = 100000
s = 5.9270
cap = x*(1+s/(12*100))**m

# print(type(d1))

date_cap_list = []
for i in range(d1.month-1, m+1):
#     print(d1.month+i)
    dd = (d1.year,
          d1.month+i,
          d1.day)
    date_cap_list.append(
        date(
            year = dd[0],
            month = dd[1],
            day = dd[2]))

# print(date_cap_list)

cap_list = []
for i in range(0, m+1):
    cap = x*(1+s/(12*100))**i
    cap = float('{:.2f}'.format(cap))
    cap_list.append(cap)


# print(
#     len(date_cap_list),
#     len(cap_list))


months_percents = pd.DataFrame({'dates': date_cap_list, 
                          'itog': cap_list})
months_percents = months_percents.set_index('dates')
print(months_percents)

                 itog
dates                
2020-01-01  100000.00
2020-02-01  100493.92
2020-03-01  100990.27
2020-04-01  101489.08
2020-05-01  101990.35
2020-06-01  102494.10
2020-07-01  103000.33
2020-08-01  103509.07
2020-09-01  104020.32
2020-10-01  104534.09
2020-11-01  105050.40


In [1390]:
df2020_only_metall = df2020[['gold', 'silver']]

In [1391]:
gold_coin_weight = 7.78
silver_coin_weight = 31.1

In [1392]:
first_gold_price_per_gramm = df2020_only_metall.iloc[1]['gold']
first_silver_price_per_gramm = df2020_only_metall.iloc[1]['silver']

In [1393]:
print(first_gold_price, first_silver_price)

3047.28 36.26


In [1394]:
gold_coin_price = float('{:.2f}'.format(gold_coin_weight * first_gold_price_per_gramm))
silver_coin_price = float('{:.2f}'.format(silver_coin_weight * first_silver_price_per_gramm))

In [1395]:
print(gold_coin_price, silver_coin_price)

23583.12 1103.43


In [1396]:
sell_gold_coin_price = float('{:.2f}'.format(gold_coin_price + (gold_coin_price * 0.07)))
sell_silver_coin_price = float('{:.2f}'.format(silver_coin_price + (silver_coin_price * 0.25)))
print(sell_gold_coin_price, sell_silver_coin_price)

25233.94 1379.29


In [1397]:
# buy coins
gold_coin_count = int(x // sell_gold_coin_price)
silver_coin_count = int(x // sell_silver_coin_price)
print(gold_coin_count, silver_coin_count)

3 72


In [1398]:
# metall in coins
gold_in_coins_gramm = gold_coin_count * gold_coin_weight
silver_in_coins_gramm = silver_coin_count * silver_coin_weight
print(gold_in_coins_count, silver_in_coins_count)

23.34 2177.0


In [1399]:
# remainders
gold_remainders = int(x % sell_gold_coin_price)
silver_remainders = int(x % sell_silver_coin_price)
print(gold_remainders, silver_remainders)

24298 691


In [1400]:
df2020_only_metall['rubles_gold'] = df2020_only_metall['gold']*gold_in_coins_gramm
df2020_only_metall['rubles_silver'] = df2020_only_metall['silver']*silver_in_coins_gramm
df2020_only_metall

<ipython-input-1400-b3bc08f1e68d>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-1400-b3bc08f1e68d>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,gold,silver,rubles_gold,rubles_silver
date,,,,
2020-01-01,3031.25,35.48,70749.3750,79446.816
2020-01-02,3031.25,35.48,70749.3750,79446.816
2020-01-03,3031.25,35.48,70749.3750,79446.816
2020-01-04,3031.25,35.48,70749.3750,79446.816
2020-01-05,3031.25,35.48,70749.3750,79446.816
...,...,...,...,...
2020-10-28,4667.69,59.68,108943.8846,133635.456
2020-10-29,4729.52,60.93,110386.9968,136434.456
2020-10-30,4759.17,60.67,111079.0278,135852.264


In [1401]:
df2020_only_metall['rubles_gold_and_remaind'] = df2020_only_metall['rubles_gold']+gold_remainders
df2020_only_metall['rubles_silver_and_remaind'] = df2020_only_metall['rubles_silver']+silver_remainders
df2020_only_metall

<ipython-input-1401-af96b25dbcfe>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-1401-af96b25dbcfe>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,gold,silver,rubles_gold,rubles_silver,rubles_gold_and_remaind,rubles_silver_and_remaind
date,,,,,,
2020-01-01,3031.25,35.48,70749.3750,79446.816,95047.3750,80137.816
2020-01-02,3031.25,35.48,70749.3750,79446.816,95047.3750,80137.816
2020-01-03,3031.25,35.48,70749.3750,79446.816,95047.3750,80137.816
2020-01-04,3031.25,35.48,70749.3750,79446.816,95047.3750,80137.816
2020-01-05,3031.25,35.48,70749.3750,79446.816,95047.3750,80137.816
...,...,...,...,...,...,...
2020-10-28,4667.69,59.68,108943.8846,133635.456,133241.8846,134326.456
2020-10-29,4729.52,60.93,110386.9968,136434.456,134684.9968,137125.456
2020-10-30,4759.17,60.67,111079.0278,135852.264,135377.0278,136543.264


In [1402]:
df2020_merge = df2020_only_metall.merge(months_percents, left_index=True, right_index=True, how='left')
# df2020_only_metall
# months_percents
df2020_merge = df2020_merge.rename(columns={'itog': "invest"})
df2020_merge

,gold,silver,rubles_gold,rubles_silver,rubles_gold_and_remaind,rubles_silver_and_remaind,invest
date,,,,,,,
2020-01-01,3031.25,35.48,70749.3750,79446.816,95047.3750,80137.816,100000.0
2020-01-02,3031.25,35.48,70749.3750,79446.816,95047.3750,80137.816,NaN
2020-01-03,3031.25,35.48,70749.3750,79446.816,95047.3750,80137.816,NaN
2020-01-04,3031.25,35.48,70749.3750,79446.816,95047.3750,80137.816,NaN
2020-01-05,3031.25,35.48,70749.3750,79446.816,95047.3750,80137.816,NaN
...,...,...,...,...,...,...,...
2020-10-28,4667.69,59.68,108943.8846,133635.456,133241.8846,134326.456,NaN
2020-10-29,4729.52,60.93,110386.9968,136434.456,134684.9968,137125.456,NaN
2020-10-30,4759.17,60.67,111079.0278,135852.264,135377.0278,136543.264,NaN


In [1403]:
df2020_merge['invest'] = df2020_merge.invest.ffill()
df2020_merge

,gold,silver,rubles_gold,rubles_silver,rubles_gold_and_remaind,rubles_silver_and_remaind,invest
date,,,,,,,
2020-01-01,3031.25,35.48,70749.3750,79446.816,95047.3750,80137.816,100000.00
2020-01-02,3031.25,35.48,70749.3750,79446.816,95047.3750,80137.816,100000.00
2020-01-03,3031.25,35.48,70749.3750,79446.816,95047.3750,80137.816,100000.00
2020-01-04,3031.25,35.48,70749.3750,79446.816,95047.3750,80137.816,100000.00
2020-01-05,3031.25,35.48,70749.3750,79446.816,95047.3750,80137.816,100000.00
...,...,...,...,...,...,...,...
2020-10-28,4667.69,59.68,108943.8846,133635.456,133241.8846,134326.456,104534.09
2020-10-29,4729.52,60.93,110386.9968,136434.456,134684.9968,137125.456,104534.09
2020-10-30,4759.17,60.67,111079.0278,135852.264,135377.0278,136543.264,104534.09


In [1731]:
trace0 = go.Scatter(x=df2020_merge.index, 
                    y=df2020_merge.invest,
                    name="Депозит",
#                     yaxis='y1',
                    marker_color = color_benchmark,
                    line={'dash': 'dash'})


trace1 = go.Scatter(x=df2020_merge.index, 
                    y=df2020_merge.rubles_gold_and_remaind,
                    name="Инвестиции в золото",
#                     yaxis='y1',
                    marker_color = color_gold
                   )


trace2 = go.Scatter(x=df2020_merge.index, 
                    y=df2020_merge.rubles_silver_and_remaind,
                    name="Инвестиции в серебро",
#                     yaxis='y1',
                    marker_color = color_silver
                   )




fig = go.Figure()

fig.add_trace(trace0)
fig.add_trace(trace1)
fig.add_trace(trace2)

fig.update_layout(
#     barmode='stack',    
#     barmode='relative',
    plot_bgcolor='rgba(0,0,0,0)',
    title_text="График инвестиций в различные инструменты",
    title_x = 0.5,
    width=1024,
    height=600,
    legend=dict(x=.45, 
                xanchor="center", 
                orientation="h"))

# Set x-axis title
fig.update_xaxes(title_text="<b>Месяцы</b>",
                 gridcolor='lightblue')

# Set y-axes titles

fig.update_yaxes(title_text="<b>руб.</b>")

fig.show()

In [1406]:
ч

,Тираж,Реализовано
y,,
2018-07-01,150000,47902
2019-07-01,100000,64425
2020-07-01,300000,174338


,Тираж,Реализовано
y,,
2018-07-01,150000,93367
2019-07-01,300000,210172
2020-07-01,500000,424876


In [1408]:
df.head()

,gold,silver,day,week,month,mean_per_week_gold,mean_per_week_silver,mean_per_month_gold,mean_per_month_silver,rolling_7_gold,...,rolling_50_gold,rolling_7_silver,rolling_35_silver,rolling_50_silver,ewm_7_gold,ewm_35_gold,ewm_50_gold,ewm_7_silver,ewm_35_silver,ewm_50_silver
date,,,,,,,,,,,,,,,,,,,,,
2018-01-01,2411.72,31.49,1,1,1,2764.575714,33.690476,2761.877527,33.423548,NaN,...,NaN,NaN,NaN,NaN,2411.72,2411.72,2411.72,31.49,31.49,31.49
2018-01-02,2411.72,31.49,2,1,1,2764.575714,33.690476,2761.877527,33.423548,NaN,...,NaN,NaN,NaN,NaN,2411.72,2411.72,2411.72,31.49,31.49,31.49
2018-01-03,2411.72,31.49,3,1,1,2764.575714,33.690476,2761.877527,33.423548,NaN,...,NaN,NaN,NaN,NaN,2411.72,2411.72,2411.72,31.49,31.49,31.49
2018-01-04,2411.72,31.49,4,1,1,2764.575714,33.690476,2761.877527,33.423548,NaN,...,NaN,NaN,NaN,NaN,2411.72,2411.72,2411.72,31.49,31.49,31.49
2018-01-05,2411.72,31.49,5,1,1,2764.575714,33.690476,2761.877527,33.423548,NaN,...,NaN,NaN,NaN,NaN,2411.72,2411.72,2411.72,31.49,31.49,31.49


In [1482]:
# fig = make_subplots(specs=[[{"secondary_y": True}]])


# trace0 = go.Scatter(x=df.index, 
#                     y=df.gold,
#                     name="Золото",
#                     line=dict(color='#f5c541'),
#                     yaxis='y1')

# trace1 = go.Bar(x=sell_silver.index, # assign x as the dataframe column 'x'
#                 y=sell_silver['Реализовано'],
#                 name = 'Реализовано', 
# #                 marker=dict(color = 'green',
# #                            line=dict(
# #                            color='red',
# #                            width = 1.5)),
#                 opacity=0.6,
#                 yaxis='y2')


# trace2 = go.Bar(x=sell_silver.index, # assign x as the dataframe column 'x'
#                 y=sell_silver['Тираж'],
#                 name = 'Тираж',
# #                 marker=dict(color = 'red',
# #                            line=dict(
# #                            color='green',
# #                            width = 1.5)),
#                 opacity=0.6,
# #                 color = 'green',
#                 yaxis='y2')


# fig.add_trace(trace0,
#               secondary_y=True)
# fig.add_trace(trace1,
#               secondary_y=False)
# fig.add_trace(trace2,
#               secondary_y=False)


# fig.update_layout(
# #     barmode='stack',    
# #     barmode='relative',
#     barmode='group',
#     plot_bgcolor='rgba(0,0,0,0)',
#     title_text="Серебро (2018 - 2020 год)",
#     title_x = 0.5,
#     width=1024,
#     height=600,
#     legend=dict(x=.45, 
#                 xanchor="center", 
#                 orientation="h"))

# # Set x-axis title
# fig.update_xaxes(title_text="<b>Годы</b>",
#                  gridcolor='lightblue')

# # Set y-axes titles

# fig.update_yaxes(title_text="<b>Цена на серебро</b> (р./грамм)", 
#                  secondary_y=True,
# #                  gridcolor='#f5c541'
#                 side='left')

# fig.update_yaxes(title_text="<b>Шт.</b>", 
#                  secondary_y=False,
#                  gridcolor='#1167b1',
#                  side='right')

# fig.show()

In [1163]:
'''
ПГ


Паш, под этой ячейкой пример того, что я бы хотел изобразить:
линия и барплоты.

Идея в том, чтобы протянуть линию по стоимости металла за три года и наложить "изготовлено / продано"

как я понимаю, проблема в том, что металл у меня - цена на каждый день, а "изготовлено / продано" - суммарно по годам.

Посмотри, пожалуйта, код из примера ниже можно ли как-то переложить, чтобыон решил задачу

'''




'\nПГ\n\n\nПаш, под этой ячейкой пример того, что я бы хотел изобразить:\nлиния и барплоты.\n\nИдея в том, чтобы протянуть линию по стоимости металла за три года и наложить "изготовлено / продано"\n\nкак я понимаю, проблема в том, что металл у меня - цена на каждый день, а "изготовлено / продано" - суммарно по годам.\n\nПосмотри, пожалуйта, код из примера ниже можно ли как-то переложить, чтобыон решил задачу\n\n'

In [1483]:
# fig = make_subplots(specs=[[{"secondary_y": True}]])#this a one cell subplot

# x = ['Shenzen', 'Australian Open', 'Fed Cup', 'Dubai', 'Indiana Wells', 'Miami', 
#                   'Stuttgart', 'Madrid', 'Rome', 'French Open', 'Birmingham', 'Wimbledon', 
#                   'Toronto', 'Cincinnati', 'US Open', 'Guangzhou', 'Wuhan', 'Beijing', 'WTA Finals'] 
# y = [280, 430, 0, 900, 1000, 390, 185, 10, 350, 70, 100, 10, 585, 585, 780, 60, 105, 10, 370]
# yy = [280, 710, 710, 1610, 2610, 3000, 3185, 3195, 3545, 3615, 3715, 3725, 4310, 4895, 5675,
#                        5735, 5840, 5850, 6220]


# trace1 = go.Bar(x=x,
#                 y=y,
#                 name='Points',            
#                 marker=dict(color= 'rgba(220,49,72, 0.8)', 
#                             line= dict(width= 1)))
 

# trace2 = go.Scatter(x=x, 
#                     y=yy, 
#                     marker= dict(line= dict(width= 1), 
#                            size= 8), 
#                     line=dict(color= '#B0122C', width= 1.5),
#                     name= 'Accumulated pts')

# #The first trace is referenced to the default xaxis, yaxis (ie. xaxis='x1', yaxis='y1')
# fig.add_trace(trace1, secondary_y=False);

# #The second trace is referenced to xaxis='x1'(i.e. 'x1' is common for the two traces) 
# #and yaxis='y2' (the right side yaxis)

# fig.add_trace(trace2, secondary_y=True);
# fig.update_layout(autosize= False, 
#                   width= 700, 
#                   height= 500, 
#                   legend= dict(x=1.15, 
#                                y=1),
#                   hovermode='x');
# fig.update_xaxes(tickangle= -45, 
#                  autorange=True,
#                  tickfont=dict(size= 10), 
#                  title= 'Tournaments', 
#                  type= 'category');

# fig.update_yaxes(range= [0, 1050], #left yaxis
#                  title= 'Points', secondary_y=False);
# fig.update_yaxes(range= [250, 6300],#right yaxis
#                  showgrid= False, 
#                  title= 'Cumulative sum of points',
#                  secondary_y=True);
# fig.show()
# # from chart_studio.plotly import  iplot
# # iplot(fig, filename='Example-Forum')

In [1166]:
#
# 1. перестроить на 2018-2020


In [1167]:
# сравнения по месяцам

In [1168]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1027 entries, 2018-01-01 to 2020-10-23
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   gold                   1027 non-null   float64
 1   silver                 1027 non-null   float64
 2   day                    1027 non-null   int64  
 3   week                   1027 non-null   UInt32 
 4   month                  1027 non-null   int64  
 5   mean_per_week_gold     1027 non-null   float64
 6   mean_per_week_silver   1027 non-null   float64
 7   mean_per_month_gold    1027 non-null   float64
 8   mean_per_month_silver  1027 non-null   float64
 9   rolling_7_gold         1021 non-null   float64
 10  rolling_35_gold        993 non-null    float64
 11  rolling_50_gold        978 non-null    float64
 12  rolling_7_silver       1021 non-null   float64
 13  rolling_35_silver      993 non-null    float64
 14  rolling_50_silver      978 non-null   

In [1045]:
# months = list(set(list(df.month)))
# months
# df[df["month"]>=1]

In [1046]:
# def plot_month_plotly(df, metall):
#     months = list(set(list(df.month)))
#     data = []
#     df = df[df["month"]>=1]
#     for month in months:
#         monthdf = df[df["month"] == month]
#         trace = go.Scatter(
#             x = monthdf['month'],
#             y = monthdf[metall],
#             mode = 'lines',
#             name = month
#         )
#         colorscale='Jet',
#         data.append(trace)
# #    iplot(data, filename='line-mode')
    
    
#         layout = go.Layout(
#         title = 'Сравнение месяц к месяцу (2020)'
#     )

#     fig = go.Figure(data=data, layout=layout)
#     iplot(fig, filename='line-mode')

In [1047]:
df2020.columns

Index(['gold', 'silver', 'day', 'week', 'month', 'mean_per_week_gold',
       'mean_per_week_silver', 'mean_per_month_gold', 'mean_per_month_silver',
       'rolling_7_gold', 'rolling_35_gold', 'rolling_50_gold',
       'rolling_7_silver', 'rolling_35_silver', 'rolling_50_silver',
       'ewm_7_gold', 'ewm_35_gold', 'ewm_50_gold', 'ewm_7_silver',
       'ewm_35_silver', 'ewm_50_silver'],
      dtype='object')

In [1048]:
def plot_month_plotly(df, metall):
    pv_test = pd.pivot_table(df,
                        index=["day"],
                        columns=["month"],
                        values=[metall])

    pv_test.columns = [col[1] for col in pv_test.columns]

    data = [go.Scatter(x = pv_test.index,
                       y = pv_test[col],
                       name = col)
            for col in pv_test.columns]

    iplot(data)

In [1049]:
plot_month_plotly(df2020, 'gold')

In [1050]:
plot_month_plotly(df2020, 'silver')

,Золото,Серебро
y,,
2018,150000,150000
2019,300000,100000
2020,500000,300000


Int64Index([2018, 2019, 2020], dtype='int64', name='y')

      Золото  Серебро
y                    
2018  150000   150000
      Золото  Серебро
y                    
2019  300000   100000
      Золото  Серебро
y                    
2020  500000   300000


['Золото', 'Серебро']

['Золото', 'Серебро'] [150000 150000]
['Золото', 'Серебро'] [300000 100000]
['Золото', 'Серебро'] [500000 300000]


In [1057]:
# нарисовать go.Pie в тоннах

In [1058]:





# for i in invest_3_years.index.unique():
#     go.Pie(labels=invest_3_years.columns, values=invest_3_years[invest_3_years.index == i])
#     fig = go.Figure(
#         go.Pie(labels=invest_3_years.columns, values=invest_3_years[invest_3_years.index == i])
#     )
#     fig.show()

In [1059]:
# график сезонов
# def metall_trend_monthly(metall, tit):
#     mycolors = ['tab:red', 'tab:blue', 'tab:green', 'tab:orange', 'tab:brown', 'tab:grey', 'tab:pink', 'tab:olive', 'deeppink', 'steelblue']
#     months = ['январь','февраль','март','апрель','май','июнь','июль','август','сентябрь','октябрь']
#     plt.figure(figsize=(16,10), dpi= 80)
#     plt.suptitle('Сравнение месячных трендов за 2020 год', fontsize=15)
#     plt.title(str(tit))
#     plt.xlabel('День месяца', fontsize=14)
#     plt.ylabel('Цена', fontsize=14)
#     lines = []
#     for i, y in enumerate(df[metall].groupby(pd.Grouper(freq='M'))):
#         p,=plt.plot(range(len(y[1])),y[1], color=mycolors[i], label=months[i])
#         lines.append(p)
#     plt.legend(handles=lines)

In [1060]:
# metall_trend_monthly('silver', 'Серебро')

In [1061]:
# metall_trend_monthly('gold', 'Золото')

In [1062]:
# from datetime import date

In [1063]:
# allbanks = pd.read_csv('docs/allbanks.csv', 
#                        encoding='cp1251', 
#                        sep=';', 
#                        index_col=0)
# allcoins = pd.read_csv('docs/allcoins.csv', 
#                        encoding='cp1251', 
#                        sep=';', 
#                        index_col=0)
# prices = pd.read_csv('docs/prices.csv', 
#                      encoding='cp1251', 
#                      sep=';')

In [1064]:
# allbanks.head()

In [1065]:
# allcoins.head()

In [1066]:
# allcoins.DT = pd.to_datetime(allcoins.DT, unit='s').dt.date
# allcoins.head()

In [1067]:
# prices.head()

In [1068]:
# prices.date = pd.to_datetime(prices.date, unit='s').dt.date
# prices.head()

In [1069]:
# result_df = prices.merge(allcoins, left_on = 'id_coin', right_index=True)
# result_df.head()

In [1070]:
# result_df = result_df.merge(allbanks, left_on = 'id_bank', right_index=True)
# result_df.head()

In [1071]:
# visualization

In [1072]:
# allcoins['metal'].unique()

In [1073]:
# nedrag_list = ['Медно-никелевый сплав',
#                'Сталь с латунным/никелевым гальваническим покрытием',
#                'Сталь с латунным гальваническим покрытием', 
#                'Латунь/Мельхиор',
#                'Сталь с никелевым гальваническим покрытием', 
#                'Мельхиор',
#                'Нейзильбер', 
#                'Медь, никель', 
#                'Латунь',
#                'Латунь/Медь, никель', 
#                'Нейзильбер/Латунь',
#                'Медь, цинк/Медь, никель']

In [1074]:
# gold_list = ['Au-999',
#              'Au-900']

In [1075]:
# silver_list = ['Ag-925', 
#                'Ag-999', 
#                'Ag-900', 
#                'Ag-500']

In [1076]:
# combine_metal_list = ['Ag-900/Au-900', 
#                       'Ag-925/Au-999', 
#                       'Ag-925/позолота', 
#                       'Pd-999', 
#                       'Pt-999', 
#                       'AU-900/Ag-925', 
#                       'Ag-925/Au-999', 
#                       'Ag-900/Au-900', 
#                       'Au-900/Ag-900']

In [1077]:
# allcoins_gold = allcoins[allcoins['metal'].isin(gold_list)]
# allcoins_gold.metal.unique()

In [1078]:
# allcoins_silver = allcoins[allcoins['metal'].isin(silver_list)]
# allcoins_silver.metal.unique()

In [1079]:
# allcoins_nedrag = allcoins[allcoins['metal'].isin(nedrag_list)]
# allcoins_nedrag.metal.unique()

In [1080]:
# allcoins_combine = allcoins[allcoins['metal'].isin(combine_metal_list)]
# allcoins_combine.metal.unique()

In [1081]:
# startdate = pd.to_datetime("2017-01-01").date()

In [1082]:
# allcoins_gold_3_years = allcoins_gold[allcoins_gold['DT'] >= startdate]
# allcoins_gold_3_years.head()

In [1083]:
# allcoins_silver_3_years = allcoins_silver[allcoins_silver['DT'] >= startdate]
# allcoins_silver_3_years.head()

In [1084]:
# allcoins_nedrag_3_years = allcoins_nedrag[allcoins_nedrag['DT'] >= startdate]
# allcoins_nedrag_3_years.head()

In [1085]:
# allcoins_combine_3_years = allcoins_combine[allcoins_combine['DT'] >= startdate]
# allcoins_combine_3_years.head()

In [1086]:
# def get_plot_count(datafr, i):
#     plt.figure(figsize=(10,5))
#     ax = sns.countplot(datafr['Nominal'])
#     ax.set(xlabel='Номинал монет', 
#            ylabel='Количество видов')
#     plt.suptitle('Количество видов монет за последние 3 года', size = 14)
# #     plt.figsize=(20, 20)
#     plt.title(str(i), size = 12)

#     plt.show()

In [1087]:
# get_plot_count(allcoins_gold_3_years, 'золото')

In [1088]:
# get_plot_count(allcoins_silver_3_years, '(серебро)')

In [1089]:
# get_plot_count(allcoins_nedrag_3_years, '(недрагоценные металлы)')

In [1090]:
# ВОТ ОТСЮДА

In [1091]:
# result_df.head()

In [1092]:
# result_df_stat = result_df[['sell_price','Nominal']]
# result_df_stat.groupby('Nominal').describe()

In [1093]:
# .groupby('Nominal').describe()

In [1094]:
# result_df_gold = result_df[result_df['metal'].isin(gold_list)]
# result_df_gold.Nominal.unique()

In [1095]:
# result_df_silver = result_df[result_df['metal'].isin(silver_list)]

In [1096]:
# result_df_nedrag = result_df[result_df['metal'].isin(nedrag_list)]

In [1097]:
# describe

In [1098]:
# result_df_gold[['sell_price','Nominal']].groupby('Nominal').describe()

In [1099]:
# result_df_silver[['sell_price','Nominal']].groupby('Nominal').describe()

In [1100]:
# result_df_nedrag[['sell_price','Nominal']].groupby('Nominal').describe()

In [1101]:
# result_df_nedrag[['sell_price','Nominal']].groupby('Nominal').describe()

In [1102]:
# result_df_nedrag[['sell_price','coin_name', 'rus_name']].groupby('sell_price').describe()

In [1103]:
# result_df_gold.info()

In [1104]:
# result_df_gold.Nominal.unique()

In [1105]:
# for i in set(result_df_gold.Nominal.unique()):
#     hidden = result_df_gold
#     hidden['Nominal'] = result_df_gold[result_df_gold['Nominal']==str(i)]
#     hidden.groupby('cat_number').describe()

In [1106]:
# result_df_gold[['sell_price','Nominal']].head()

In [1107]:
# res_clear = result_df_gold[['sell_price','Nominal']]
# res_clear = res_clear[res_clear['sell_price']>30000] #мусорные цены, надо вычистить в базе
# nominal_list = set(res_clear['Nominal'])

In [1108]:
# nominal_list

In [1109]:
# for i in nominal_list:
#     hidden_df = res_clear[res_clear['Nominal']==i]
#     sns.histplot(hidden_df['sell_price'], 
#                 color="gold",
#                 bins = 10).set(title=i)
#     plt.show()

In [1110]:
# res_clear = result_df_silver[['sell_price','Nominal']]
# res_clear = res_clear[res_clear['sell_price']>2000] #мусорные цены, надо вычистить в базе
# nominal_list = set(res_clear['Nominal'])

In [1111]:
# for i in nominal_list:
#     hidden_df = res_clear[res_clear['Nominal']==i]
#     sns.histplot(hidden_df['sell_price'], 
#                 color="silver",
#                 bins=100).set(title=i)
#     plt.show()

In [ ]:
# df.to_csv('docs/drags.csv')